## Distribution strategies using Keras

In [1]:
import tensorflow as tf

### Simulate an environment with 4 GPUs 

Don't run these cells when using an ec2 instance

In [2]:
physical_GPUS = tf.config.list_physical_devices(device_type='GPU')
physical_GPUS

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')


tf.config.set_logical_device_configuration(
  physical_devices[0],
  [tf.config.LogicalDeviceConfiguration(memory_limit=2000),
    tf.config.LogicalDeviceConfiguration(memory_limit=2000),
    tf.config.LogicalDeviceConfiguration(memory_limit=2000),
    tf.config.LogicalDeviceConfiguration(memory_limit=2000)])

logical_devices = tf.config.list_logical_devices('GPU')
print('logical_devices', logical_devices)

logical_devices [LogicalDevice(name='/device:GPU:0', device_type='GPU'), LogicalDevice(name='/device:GPU:1', device_type='GPU'), LogicalDevice(name='/device:GPU:2', device_type='GPU'), LogicalDevice(name='/device:GPU:3', device_type='GPU')]


## Manually placing operations and variables on different GPUS

In [6]:
with tf.device(logical_devices[0]):

  # Create some tensors
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

  # Operation inside the context will run in the device associated
  c = tf.matmul(a, b)
  print('c', c)

with tf.device(logical_devices[1]):

  # Create some tensors
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
  c = tf.matmul(a, b)
  print('c', c)


strategy = tf.distribute.MirroredStrategy(["GPU:0", "GPU:1"])
with strategy.scope():
  # Create some tensors
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
  c = tf.matmul(a, b)
  print('c', c)
  print("mirrorred variable 'a'", a)




c tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)
c tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


c tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)
mirrorred variable 'a' MirroredVariable:{
  0: <tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>,
  1: <tf.Variable 'Variable/replica_1:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>
}


### Get any dummy dataset for training

In this case I got the imagenette dataset (small Imagenet)

In [7]:
import tensorflow as tf
import tensorflow_datasets as tfds
from  tensorflow.data.experimental import AUTOTUNE



def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label


ds, info = tfds.load('imagenette',  split=['train', 'validation'],shuffle_files=True,  as_supervised=True, with_info=True,)
ds_train,  ds_test = ds


ds_train = ds_train.map(lambda x,y : (tf.image.resize(x, (128,128) ),y), num_parallel_calls= AUTOTUNE).map(normalize_img, num_parallel_calls= AUTOTUNE ).batch(256).prefetch(AUTOTUNE)
ds_test = ds_test.map(lambda x,y : (tf.image.resize(x, (128,128) ),y), num_parallel_calls= AUTOTUNE).map(normalize_img,  num_parallel_calls= AUTOTUNE).batch(256).prefetch(AUTOTUNE)

Shuffling and writing examples to /root/tensorflow_datasets/imagenette/full-size-v2/0.1.0.incompleteIY6JFS/imagenette-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imagenette/full-size-v2/0.1.0.incompleteIY6JFS/imagenette-validation.tfrecord


Dataset imagenette downloaded and prepared to /root/tensorflow_datasets/imagenette/full-size-v2/0.1.0. Subsequent calls will reuse this data.


## Training a model with a strategy using a single device

tf.distribute.OneDeviceStrategy is a strategy to place all variables and computation on a single specified device.

This strategy is distinct from the Default Strategy in a number of ways. In the Default Strategy, the variable placement logic remains unchanged when compared to running TensorFlow without any distribution strategy. But when using OneDeviceStrategy, all variables created in its scope are explicitly placed on the specified device. Moreover, any functions called via OneDeviceStrategy.run will also be placed on the specified device.

In [8]:
### here
strategy = tf.distribute.OneDeviceStrategy(device=logical_devices[0])
with strategy.scope():
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(128, 128, 3)),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(10)
  ])
  model.compile(
      optimizer=tf.keras.optimizers.Adam(0.001),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
  )

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
37/37 [==============================] - 24s 634ms/step - loss: 7.9282 - sparse_categorical_accuracy: 0.1404 - val_loss: 2.3055 - val_sparse_categorical_accuracy: 0.1972
Epoch 2/6
37/37 [==============================] - 23s 618ms/step - loss: 2.0455 - sparse_categorical_accuracy: 0.2930 - val_loss: 1.9916 - val_sparse_categorical_accuracy: 0.2831
Epoch 3/6
37/37 [==============================] - 23s 614ms/step - loss: 1.9002 - sparse_categorical_accuracy: 0.3465 - val_loss: 1.9421 - val_sparse_categorical_accuracy: 0.3276
Epoch 4/6
37/37 [==============================] - 23s 614ms/step - loss: 1.8683 - sparse_categorical_accuracy: 0.3637 - val_loss: 1.9075 - val_sparse_categorical_accuracy: 0.3529
Epoch 5/6
37/37 [==============================] - 23s 615ms/step - loss: 1.8473 - sparse_categorical_accuracy: 0.3742 - val_loss: 1.9150 - val_sparse_categorical_accuracy: 0.3434
Epoch 6/6
37/37 [==============================] - 23s 614ms/step - loss: 1.8359 - sparse_categorica

## Training a model with mirrored strategy

A replica of the model is created in each GPU and the all reduce operation (Nvidia Collective Communications Library) is used to keep the models synchronized

In [9]:
import tensorflow as tf
import tensorflow_datasets as tfds

### here
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(128, 128, 3)),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(10)
  ])
  model.compile(
      optimizer=tf.keras.optimizers.Adam(0.001),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
  )

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').


37/37 [==============================] - 28s 656ms/step - loss: 7.2196 - sparse_categorical_accuracy: 0.1541 - val_loss: 2.7073 - val_sparse_categorical_accuracy: 0.1929
Epoch 2/6
37/37 [==============================] - 23s 621ms/step - loss: 2.1891 - sparse_categorical_accuracy: 0.2748 - val_loss: 2.0358 - val_sparse_categorical_accuracy: 0.3006
Epoch 3/6
37/37 [==============================] - 23s 624ms/step - loss: 2.0041 - sparse_categorical_accuracy: 0.3156 - val_loss: 2.1517 - val_sparse_categorical_accuracy: 0.2917
Epoch 4/6
37/37 [==============================] - 23s 619ms/step - loss: 1.9772 - sparse_categorical_accuracy: 0.3340 - val_loss: 2.2020 - val_sparse_categorical_accuracy: 0.2935
Epoch 5/6
37/37 [==============================] - 23s 633ms/step - loss: 2.0566 - sparse_categorical_accuracy: 0.3259 - val_loss: 2.0081 - val_sparse_categorical_accuracy: 0.3580
Epoch 6/6
37/37 [==============================] - 23s 627ms/step - loss: 1.8764 - sparse_categorical_accuracy